In [1]:
from lxml import html
import urllib
from urllib import request
from bs4 import BeautifulSoup
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import requests
import lxml.html
from requests import get
import pandas as pd
import numpy as np
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / size)     
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

## Vostok Vesti

In [9]:
months_2018 = ['11','12']
months_2019 = ['1','2']
days_1 = ['1','2','3','4','5','6','7','8','9']
days_2 = range(10,32)
days_2 = ["{:02d}".format(x) for x in days_2]
days = days_1 + days_2
categories = ['1','2','7','6','3','17','8','10']

In [ ]:
vostok_urls_2018 = []
for a in categories:
    for i in months_2018: 
        for y in log_progress(days):
            try:
                url = lxml.html.fromstring(requests.get('https://www.vostok.rs/index.php?catnovosti=' + a + '&day=' + y + '&month=' + i + '&year=2018' + '-1&option=btg_arhiva&count=0&akcija=datum').content)
                vostok_urls_2018.append(url)
            except:
                pass

In [ ]:
vostok_urls_2019 = []
for a in categories:
    for i in months_2019: 
        for y in log_progress(days):
            try: 
                url = lxml.html.fromstring(requests.get('https://www.vostok.rs/index.php?catnovosti=' + a + '&day=' + y + '&month=' + i + '&year=-1' + '-1&option=btg_arhiva&count=0&akcija=datum').content)
                vostok_urls_2019.append(url)
            except:
                pass

In [15]:
vostok_urls = vostok_urls_2018 + vostok_urls_2019

Get only relevant links - those containing idnovost are actual articles. Those containing javascript are not articles. 

In [ ]:
relevant_links = [x.xpath('//a/@href') for x in vostok_urls]
clean_links = [x for y in relevant_links for x in y if 'idnovost' in x and 'javascript' not in x]
clean_links = ['https://www.vostok.rs' + x for x in clean_links]

In [22]:
clean_links = list(set(clean_links))
len(clean_links)
np.save("clean_links_vostok.npy", clean_links)

In [141]:
categories = []
for x in clean_links:
    string = re.findall('catnovosti=(.*?)&idnovost', x)
    categories.append(string) 
len(categories)

3837

In [ ]:
pages = []
for page in log_progress(clean_links): 
    new_pages = requests.get(page)
    pages.append(new_pages)
trees = [page.content for page in pages]
soups = [BeautifulSoup(tree, 'lxml') for tree in trees]
np.save("vostok_soups.npy", trees)

In [ ]:
titles = []
for soup in log_progress(soups):
    try:
        title = soup.find("div", {"class" : "col-sm-12 col-xs-12 news_main_headline"}).findAll('h1')
        titles.append(title)
    except AttributeError:
        titles.append('problem')
texts_and_dates = []
for soup in log_progress(soups):
    try:
        text_date = soup.select('div span')
        texts_and_dates.append(text_date)
    except AttributeError:
        texts_and_dates.append("problem")

In [170]:
all_data = pd.DataFrame(
    {'title': titles,
     'texts_and_dates' : texts_and_dates,
     'categories' : categories,
    })

In [173]:
all_data.isna().sum()

title              0
texts_and_dates    0
categories         0
dtype: int64

In [174]:
all_data.to_csv('vostok.csv', sep='\t', encoding='utf-8')

## Sputnik 

In [148]:
days = range(32)
days = ["{:02d}".format(x) for x in days]
del days[0]
categories = ['svet', 'politika', 'ekonomija', 'kultura', 'naoruzanje']
months_2018 = ['11', '12']
months_2019 = ['01', '02']

In [ ]:
sputnik_urls_2018 = []
for a in categories:
    for i in months_2018:
        for y in log_progress(days): 
            try: 
                urls = lxml.html.fromstring(requests.get('https://rs.sputniknews.com/' + a + '/2018'+ i + y).content)
                sputnik_urls_2018.append(urls)
            except:
                pass

In [ ]:
sputnik_urls_2019 = []
for a in categories:
    for i in months_2019:
        for y in log_progress(days): 
            try: 
                urls = lxml.html.fromstring(requests.get('https://rs.sputniknews.com/' + a + '/2019'+ i + y).content)
                sputnik_urls_2019.append(urls)
            except:
                pass

In [156]:
sputnik_urls = sputnik_urls_2018 + sputnik_urls_2019

Only pick up actual articles. When length shorter than 19, the links are calendar archive links, when links contain export they are not articles. When they contain politika they are articles about politics. 

In [158]:
relevant_links = [x.xpath('//a/@href') for x in sputnik_urls]
clean_links = []
crap_links = []
for y in relevant_links:
    for x in y: 
        for i in categories:
            if i in x and 'https' not in x and 'export' not in x and len(x) > 19:
                clean_links.append(x)
            else: 
                crap_links.append(x)
clean_links = ['https://rs.sputniknews.com' + x for x in clean_links]

In [163]:
clean_links = list(set(clean_links))
len(clean_links)
np.save("clean_links_sputnik.npy", clean_links)
clean_links = np.load('clean_links_sputnik.npy')

In [ ]:
pages = []
for page in log_progress(clean_links): 
    new_pages = requests.get(page)
    pages.append(new_pages)
trees = [page.content for page in pages]
soups = [BeautifulSoup(tree, 'lxml') for tree in trees]
np.save("sputnik_soups.npy", soups)

In [ ]:
titles = []
for soup in log_progress(soups):
    try:
        title = soup.find("div", {"class" : "b-article__header-title"}).findAll('h1')
        titles.append(title)
    except AttributeError:
        titles.append('problem')
lead_texts = []
for soup in log_progress(soups):
    try:
        lead_text = soup.find("div", {"class" : "b-article__lead"}).findAll('p')
        lead_texts.append(lead_text)
    except AttributeError:
        lead_texts.append('problem')
texts = []
for soup in log_progress(soups):
    try:
        text = soup.find("div", {"class" : "b-article__text"}).findAll('p')
        texts.append(text)
    except AttributeError
        texts.append('problem')
dates = []
for soup in log_progress(soups):
    try:
        date = soup.find("meta", itemprop = "dateCreated", content=True)['content']
        dates.append(date)
    except AttributeError:
        dates.append('problem')
categories = []
for soup in log_progress(soups):
    try:
        category = soup.find('a', {'itemprop' : "articleSection"})
        categories.append(category)
    except AttributeError:
        category.append('problem')
categories = [x for x in categories if str(x) != 'None']

In [201]:
all_data = pd.DataFrame(
    {'title': titles,
     'date' : dates,
     'lead_text': lead_texts,
     'text': texts,
     'category' : categories
    })

In [203]:
all_data.isna().sum()

title        0
date         0
lead_text    0
text         0
category     0
dtype: int64

In [204]:
all_data.to_csv('sputnik.csv', sep='\t', encoding='utf-8')

In [205]:
all_data = pd.read_csv('sputnik.csv', sep = '\t')